In [ ]:
import math
import numpy as np
from pathlib import Path
from tqdm import tqdm
from typing import List, Any
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings, HuggingFaceInstructEmbeddings
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [ ]:
docs = []
metadata = []
for p in Path("./datasets/huggingface_docs/").iterdir():
    if not p.is_dir():
        with open(p) as f:
            # the first line is the source of the text
            source = f.readline().strip().replace('source: ', '')
            docs.append(f.read())
            metadata.append({"source": source})

print(f'number of documents: {len(docs)}')

In [ ]:
text_splitter = CharacterTextSplitter(
    separator="",
    chunk_size=812,
    chunk_overlap=100,
    length_function=len,
)
docs = text_splitter.create_documents(docs, metadata)
print(f'number of chunks: {len(docs)}')

In [ ]:
model_name = "hkunlp/instructor-large"
embed_instruction = "Represent the Hugging Face library documentation"
query_instruction = "Query the most relevant piece of information from the Hugging Face documentation"

# embedding_model = HuggingFaceInstructEmbeddings(
#     model_name=model_name,
#     embed_instruction=embed_instruction,
#     query_instruction=query_instruction,
# )

In [ ]:
class AverageInstructEmbeddings(HuggingFaceInstructEmbeddings):
    max_length: int = None

    def __init__(self, max_length: int = 512, **kwargs: Any):
        super().__init__(**kwargs)
        self.max_length = max_length
        if self.max_length < 0:
            print('max_length is not specified, using model default max_seq_length')

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        all_embeddings = []
        for text in tqdm(texts, desc="Embedding documents"):
            if len(text) > self.max_length and self.max_length > -1:
                n_chunks = math.ceil(len(text)/self.max_length)
                chunks = [
                    text[i*self.max_length:(i+1)*self.max_length]
                    for i in range(n_chunks)
                ]
                instruction_pairs = [[self.embed_instruction, chunk] for chunk in chunks]
                chunk_embeddings = self.client.encode(instruction_pairs)
                avg_embedding = np.mean(chunk_embeddings, axis=0)
                all_embeddings.append(avg_embedding.tolist())
            else:
                instruction_pairs = [[self.embed_instruction, text]]
                embeddings = self.client.encode(instruction_pairs)
                all_embeddings.append(embeddings[0].tolist())

        return all_embeddings


embedding_model = AverageInstructEmbeddings(  
    model_name=model_name,
    embed_instruction=embed_instruction,
    query_instruction=query_instruction,
    max_length=512,
)

In [ ]:
embeddings = embedding_model.embed_documents(texts=[d.page_content for d in docs[:10]])

In [ ]:
index = FAISS.from_documents(docs, embedding_model)

In [ ]:
index.save_local('../indexes/index-large-notebooks/')

In [ ]:
index = FAISS.load_local(f'../indexes/index-large-notebooks/', embedding_model)
docs = index.similarity_search(query='how to create a pipeline object?', k=5)
docs[0].page_content
docs[0].metadata

In [ ]:
for index, doc in enumerate(docs, start=1):
    print(f"\n{'='*100}\n")
    print(f"Document {index} of {len(docs)}")
    print("Page Content:")
    print(f"\n{'-'*100}\n")
    print(doc.page_content, '\n')
    print(doc.metadata)

In [ ]:
from huggingface_hub import HfApi

index_name = 'index-large-notebooks'

api = HfApi()
api.create_repo(
    repo_id=f'KonradSzafer/{index_name}',
    repo_type='dataset',
    private=False,
    exist_ok=True
)
api.upload_folder(
    folder_path=f'../indexes/{index_name}',
    repo_id=f'KonradSzafer/{index_name}',
    repo_type='dataset',
)